In [19]:
import numpy as numpy
import pandas as pd
import seaborn as sns #for plotting
import matplotlib.pyplot as plt
import random
import os
from scipy.stats import zscore
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import requests
from Bio.PDB import PDBParser, NeighborSearch
from propy.PseudoAAC import GetPseudoAAC, _Hydrophobicity, _hydrophilicity, _residuemass
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import glob
from propy.PseudoAAC import GetPseudoAAC, _Hydrophobicity, _hydrophilicity, _residuemass
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from scipy.stats import chi2_contingency

In [20]:
# Wasser aus ctla4_interface_residues entfernen
# Datei laden
df = pd.read_csv("data/ctla4/ctla4_interface_residues.csv")

# Zeilen mit 'HOH' in einer beliebigen Spalte entfernen (z. B. in Spalte 3)
df_clean = df[~df.iloc[:, 2].astype(str).str.contains("HOH")]

# Gesäuberte Datei speichern
df_clean.to_csv("data/ctla4/ctla4_interface_residues_clean.csv", index=False)

In [21]:
files = {
    'CTLA4': 'data/ctla4/ctla4_epitope_residues.csv',
    'KIR': 'data/kir/kir_epitope_residues.csv',
    'LAG3': 'data/lag3/lag3_epitope_residues.csv',
    'PD1': 'data/pd1/pd1_epitope_residues.csv',
    'PDL1': 'data/pdl1/pdl1_epitope_residues.csv',
    'TIM3': 'data/tim3/tim3_epitope_residues.csv',
}

# Häufigkeitstabellen erstellen
dfs = []
for checkpoint, path in files.items():
    df = pd.read_csv(path)
    counts = df['residue_name'].value_counts()
    counts.name = checkpoint
    dfs.append(counts)

# Kontingenztabelle erzeugen (Residuen = Spalten, Checkpoints = Zeilen)
contingency_table = pd.DataFrame(dfs).fillna(0).astype(int)

print("Kontingenztabelle:\n", contingency_table)

# Chi²-Test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("\nChi²-Wert:", chi2)
print("p-Wert:", p)
print("Freiheitsgrade:", dof)

# Erwartete Häufigkeiten und Residuen
expected_df = pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns)
residuals = (contingency_table - expected_df) / (expected_df**0.5)

print("\nStandardisierte Residuen:\n", residuals)


Kontingenztabelle:
 residue_name  TYR  PRO  GLU  LEU  MET  THR  ILE  LYS  VAL  GLN  ...  GLY  SER  \
CTLA4          35   32   31   29   22   18   18   13   11   11  ...   10    8   
KIR             2    1    0    4    0    1    2    2    0    0  ...    1    2   
LAG3            1    4    4    1    0    0    0    0    2    5  ...    3    7   
PD1             8   37   26   21    2   13   30   31   15   27  ...    5   43   
PDL1           22    0   16    5   15    3   11    8   14    7  ...   11    6   
TIM3            0    3    2    0    1    0    3    2    2    1  ...    3    0   

residue_name  ALA  HIS  ASN  ASP  PHE  CYS  TRP  NAG  
CTLA4           7    6    3    2    2    1    0    0  
KIR             0    3    0    2    2    0    0    0  
LAG3            3    0    0    0    2    0    0    0  
PD1            36    0   17   21   15    0    3    1  
PDL1           21    7    6   22    3    0    1    0  
TIM3            1    0    2    1    1    2    0    0  

[6 rows x 21 columns]

Chi